In [1]:
import nltk

from nltk.util import ngrams

from nltk.stem import WordNetLemmatizer

In [2]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [3]:
import re
import pandas as pd
import numpy as np

In [4]:
file="D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\sentiment labelled sentences\\yelp_labelled.txt"

In [5]:
with open(file,'r') as f:
    review_lines=f.readlines()

In [6]:
len(review_lines)

1000

In [7]:
#for one_line in review_lines:
    #print(one_line)

reviews = []
review_labels = []
for line in review_lines:
    review=line.split('\t')[0]
    reviews.append(review)
    label=line.split('\t')[1]
    review_labels.append(int(label))

In [8]:
review_labels[0]

1

In [9]:
review_string = ' '.join(reviews)

In [10]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
review_tokens = tknzr.tokenize(review_string)

In [11]:
len(review_tokens)

12471

#### Removing punctuation

In [12]:
punctuation = re.compile(r'[-.,:;()"?!|0-9]')

In [13]:
review_token = []
for word in review_tokens:
    word = punctuation.sub('',word)
    if len(word)>0:
        review_token.append(word)


In [14]:
print(len(review_token))

10852


#### Removing stopwords

In [15]:
from nltk.corpus import stopwords, wordnet

In [16]:
stopwords

<WordListCorpusReader in 'C:\\Users\\Amrendra\\AppData\\Roaming\\nltk_data\\corpora\\stopwords'>

In [17]:
stop_words=set(stopwords.words('english'))

In [18]:
review_tokens3=[]
for word in review_token:
    word=word.lower()
    if word not in stop_words:
        review_tokens3.append(word)

In [19]:
len(review_tokens3)

5592

#### Finding frequency distribution

In [20]:
from nltk.probability import FreqDist
fdist = FreqDist(review_tokens3)
#fdist = FreqDist()
#for word in review_tokens3:
#    fdist[word] += 1

In [21]:
type(fdist)

nltk.probability.FreqDist

In [22]:
fdist.most_common(10)

[('food', 126),
 ('place', 106),
 ('good', 95),
 ('service', 84),
 ('great', 70),
 ('back', 61),
 ('like', 46),
 ('go', 43),
 ('time', 42),
 ('really', 36)]

#### Part of speech tag

In [23]:
pos_list = []
for word in review_tokens3:
    pos_list.append(nltk.pos_tag([word]))
    

In [24]:
pos1=pos_list[0]
pos1

[('wow', 'NN')]

In [25]:
pos_set = set()
for pos in pos_list:
    pos_set.add(pos[0][1])

In [26]:
pos_set

{'$',
 "''",
 'CC',
 'CD',
 'DT',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'MD',
 'NN',
 'NNS',
 'PRP',
 'RB',
 'RBR',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBZ'}

#### Finding Adjectives

In [27]:
pos_JJ = []
for pos in pos_list:
    if pos[0][1] in ['JJ','JJR','JJS']:
        pos_JJ.append(pos[0][0])

In [28]:
len(pos_JJ)

708

In [29]:
fdist_JJ = FreqDist()
for word in pos_JJ:
    fdist_JJ[word] += 1

In [30]:
fdist_JJ.most_common(10)

[('good', 95),
 ('great', 70),
 ('best', 30),
 ('nice', 25),
 ('delicious', 23),
 ('disappointed', 19),
 ('bad', 18),
 ('much', 18),
 ('worst', 15),
 ('fresh', 14)]

In [31]:
positive_review = []
negative_review = []

for review in review_lines:
    review_list=review.split('\t')
    
    label= int(review_list[1])

    if label == 1:
        pos_review = review_list[0]
        pos_review = pos_review.lower()
        positive_review.append(pos_review)
    
    elif label == 0:
        neg_review = review_list[0]
        neg_review = neg_review.lower()
        negative_review.append(neg_review)   

#### Working on positive words

In [32]:
pos_review_string = ' '.join(positive_review)

In [33]:
pos_review_tokens= nltk.word_tokenize(pos_review_string)

In [34]:
pos_rev_trigram= list(nltk.trigrams(pos_review_tokens))

In [35]:
freq_dist_pos_review=nltk.FreqDist(pos_rev_trigram)
freq_dist_pos_review.most_common(10)

[(('!', '!', '!'), 46),
 (('this', 'place', 'is'), 12),
 (('the', 'food', 'was'), 11),
 (('this', 'place', '.'), 9),
 (('.', 'the', 'food'), 7),
 (('our', 'server', 'was'), 7),
 (('.', 'this', 'place'), 7),
 (('.', 'i', 'love'), 7),
 (('and', 'it', 'was'), 6),
 (('.', 'this', 'is'), 6)]

#### Working on negative words

In [36]:
neg_review_string=' '.join(negative_review)
neg_review_tokens=nltk.word_tokenize(neg_review_string)
neg_review_trigram= list(nltk.trigrams(neg_review_tokens))
freq_dist_neg_review=nltk.FreqDist(neg_review_trigram)
freq_dist_neg_review.most_common(10)

[(('wo', "n't", 'be'), 12),
 (('.', 'it', 'was'), 11),
 (('the', 'service', 'was'), 10),
 (('i', 'do', "n't"), 10),
 (('!', '!', '!'), 10),
 (('.', 'this', 'place'), 9),
 (('the', 'food', 'was'), 9),
 (('.', 'the', 'service'), 8),
 (('i', 'wo', "n't"), 8),
 ((',', 'but', 'i'), 7)]

#### Predicting reviews using Machine Learning

In [37]:
review_list=[]
for review in review_lines:
    reviews=review.split('\t')[0]
    review=review.lower()
    review_list.append(reviews)

In [41]:
type(review_list)

list

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vect= TfidfVectorizer()
x_tfidf=tf_vect.fit_transform(review_list)

In [43]:
x_tfidf.shape

(1000, 2036)

In [44]:
x_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
tf_vect.get_feature_names()

['00',
 '10',
 '100',
 '11',
 '12',
 '15',
 '17',
 '1979',
 '20',
 '2007',
 '23',
 '30',
 '30s',
 '35',
 '40',
 '40min',
 '45',
 '4ths',
 '5lb',
 '70',
 '85',
 '90',
 '99',
 'about',
 'above',
 'absolute',
 'absolutely',
 'absolutley',
 'accident',
 'accommodations',
 'accomodate',
 'accordingly',
 'accountant',
 'ache',
 'acknowledged',
 'across',
 'actual',
 'actually',
 'added',
 'affordable',
 'after',
 'afternoon',
 'again',
 'ago',
 'ahead',
 'airline',
 'airport',
 'ala',
 'albondigas',
 'all',
 'allergy',
 'almonds',
 'almost',
 'alone',
 'also',
 'although',
 'always',
 'am',
 'amazing',
 'ambiance',
 'ambience',
 'amount',
 'ample',
 'an',
 'and',
 'andddd',
 'angry',
 'annoying',
 'another',
 'anticipated',
 'any',
 'anymore',
 'anyone',
 'anything',
 'anytime',
 'anyway',
 'anyways',
 'apart',
 'apologize',
 'apology',
 'app',
 'appalling',
 'apparently',
 'appealing',
 'appetite',
 'appetizer',
 'appetizers',
 'apple',
 'approval',
 'are',
 'area',
 'aren',
 'arepas',
 'ar

In [46]:
x_tf=pd.DataFrame(x_tfidf.toarray(),columns=tf_vect.get_feature_names())

In [47]:
x_tf.head()

,00,10,100,11,12,15,17,1979,20,2007,...,yelpers,yet,you,your,yourself,yucky,yukon,yum,yummy,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
y=review_labels

In [49]:
x_tf_train, x_tf_test, y_tf_train,y_tf_test = train_test_split(x_tf,y, test_size=0.2,random_state=3)

In [50]:
x_tf_train.shape

(800, 2036)

In [51]:
x_tf_test.shape

(200, 2036)

In [52]:
from sklearn.naive_bayes import MultinomialNB

clf_tf = MultinomialNB()
clf_tf.fit(x_tf_train,y_tf_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [53]:
y_tf_pred= clf_tf.predict(x_tf_test)

In [54]:
metrics.accuracy_score(y_tf_pred,y_tf_test)

0.795

In [55]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_tf_pred,y_tf_test)

array([[77, 31],
       [10, 82]], dtype=int64)

In [56]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
skf = StratifiedKFold(n_splits=10)
params = {}
nb = MultinomialNB()
gs = GridSearchCV(nb, cv=skf, param_grid=params, return_train_score=True)


In [57]:
gs.fit(x_tf_train,y_tf_train)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
       error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [58]:
gs.cv_results_

{'mean_fit_time': array([0.02985396]),
 'std_fit_time': array([0.04119695]),
 'mean_score_time': array([0.00099628]),
 'std_score_time': array([0.00044601]),
 'params': [{}],
 'split0_test_score': array([0.83950617]),
 'split1_test_score': array([0.7654321]),
 'split2_test_score': array([0.83950617]),
 'split3_test_score': array([0.825]),
 'split4_test_score': array([0.775]),
 'split5_test_score': array([0.7875]),
 'split6_test_score': array([0.7875]),
 'split7_test_score': array([0.73417722]),
 'split8_test_score': array([0.82278481]),
 'split9_test_score': array([0.7721519]),
 'mean_test_score': array([0.795]),
 'std_test_score': array([0.03352586]),
 'rank_test_score': array([1]),
 'split0_train_score': array([0.9527121]),
 'split1_train_score': array([0.94993046]),
 'split2_train_score': array([0.95549374]),
 'split3_train_score': array([0.95972222]),
 'split4_train_score': array([0.95972222]),
 'split5_train_score': array([0.95972222]),
 'split6_train_score': array([0.95972222]),


In [6]:
import nltk
sentence = "the little yellow dog barked at the cat"
#Define your grammar using regular expressions
grammar = r'''
    NP: {<DT>?<JJ>*<NN>} # NP
    '''

In [7]:
chunkParser = nltk.RegexpParser(grammar)
tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
tagged

[('the', 'DT'),
 ('little', 'JJ'),
 ('yellow', 'JJ'),
 ('dog', 'NN'),
 ('barked', 'VBD'),
 ('at', 'IN'),
 ('the', 'DT'),
 ('cat', 'NN')]

In [8]:
tree = chunkParser.parse(tagged)

In [9]:
for subtree in tree.subtrees():
    print(subtree)

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))
(NP the/DT little/JJ yellow/JJ dog/NN)
(NP the/DT cat/NN)


In [10]:
tree.draw()